Url: https://tbrain.trendmicro.com.tw/Competitions/Details/2

In [26]:
#import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import Row
from pyspark.sql.functions import col, udf, lag, rank, lit
from pyspark.sql.window import Window

In [27]:
global Path
global DoEval #是否進行模型評估

DoEval = False
next_date_range = ["20180430", "20180501", "20180502", "20180503", "20180504"] #設定預測區間
ignore_dates = ["20180501"]#設定排除日(如端午節)
predict_start_date = next_date_range[0]

if sc.master[0:5]=="local":
    #Path = "file:/c:/D Drive/work/bigData/pySpark/TBrain_Round2_DataSet_20180504"
    #Path = "file:/Users/yungchuanlee/Documents/learn/AI競賽/ETF預測/TBrain_Round2_DataSet_20180504"
    Path = "file:/home/hduser/app/bigdata/competition/etf/TBrain_Round2_DataSet_20180504"
else:
    Path = "hdfs://master:9000/user/hduser"

In [28]:
sc.master

'local[*]'

In [29]:
#define alias of columns
col_alias_etf= {"代碼":"etf_id", "日期": "etf_date", "中文簡稱": "etf_name", "開盤價(元)":"etf_open", 
            "最高價(元)":"etf_high", "最低價(元)":"etf_low", "收盤價(元)":"etf_close", "成交張數(張)":"etf_count"}
col_alias_stock= {"代碼":"stock_id", "日期": "stock_date", "中文簡稱": "stock_name", "開盤價(元)":"stock_open", 
            "最高價(元)":"stock_high", "最低價(元)":"stock_low", "收盤價(元)":"stock_close", "成交張數(張)":"stock_count"}

In [30]:
#udf
def to_double(str_val):
    return float(str_val.replace(",",""))
to_double=udf(to_double)

In [31]:
#def function to read data (因檔案格式都相同)
def read_data(file_name, col_alias):
    str_cols = ["代碼","日期", "中文簡稱"]
    raw_data = spark.read.option("encoding", "Big5").csv(Path + "/" + file_name, header=True, sep=",")
    print("Total " + file_name + " count: " + str(raw_data.count()))
    #rename cols and correct type 
    num_cols = [col_name for col_name in raw_data.columns if col_name not in str_cols]
    final_data=raw_data.select( [col(str_col_name).alias(col_alias[str_col_name]) for str_col_name in str_cols] + 
                                  [to_double(col(num_col_name)).cast("double").alias(col_alias[num_col_name]) for num_col_name in num_cols] )
    final_data.printSchema()
    final_data.show(5)
    return final_data

In [32]:
print("starting import tetfp.csv(台灣18檔ETF股價資料)...")
tetfp_dt=read_data("tetfp.csv", col_alias_etf)

starting import tetfp.csv(台灣18檔ETF股價資料)...
Total tetfp.csv count: 19126
root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- etf_name: string (nullable = true)
 |-- etf_open: double (nullable = true)
 |-- etf_high: double (nullable = true)
 |-- etf_low: double (nullable = true)
 |-- etf_close: double (nullable = true)
 |-- etf_count: double (nullable = true)

+-------+--------+----------------+--------+--------+-------+---------+---------+
| etf_id|etf_date|        etf_name|etf_open|etf_high|etf_low|etf_close|etf_count|
+-------+--------+----------------+--------+--------+-------+---------+---------+
|0050   |20130102|元大台灣50          |    54.0|   54.65|   53.9|     54.4|  16487.0|
|0050   |20130103|元大台灣50          |    54.9|   55.05|  54.65|    54.85|  29020.0|
|0050   |20130104|元大台灣50          |   54.85|   54.85|   54.4|     54.5|   9837.0|
|0050   |20130107|元大台灣50          |   54.55|   54.55|   53.9|    54.25|   8910.0|
|0050   |20130108|元大台灣50     

In [33]:
#EDA
#range of date
tetfp_dt.describe('etf_date').show()

+-------+-------------------+
|summary|           etf_date|
+-------+-------------------+
|  count|              19126|
|   mean|2.015339706190526E7|
| stddev| 15776.777847910937|
|    min|           20130102|
|    max|           20180504|
+-------+-------------------+



In [34]:
# print("starting import taetfp.csv(台灣18檔ETF調整後股價資料)...")
# taetfp_dt=read_data("taetfp.csv", col_alias_etf)

In [35]:
# print("starting import tsharep.csv(台灣個股股價資料)...")
# tsharep_dt=read_data("tsharep.csv", col_alias_stock)

In [36]:
# print("starting import tasharep.csv(台灣個股調整後股價資料)...")
# tasharep_dt=read_data("tasharep.csv", col_alias_stock)

In [37]:
import sys
from pyspark.sql.functions import lag, col, avg,collect_list, lit
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, DoubleType, IntegerType
#declare previous row windows
wsSpec_etf = Window.partitionBy('etf_id').orderBy('etf_date') #time window for normal case
wsSpec_etf_close_price_raw = Window.partitionBy('etf_id').orderBy('row_idx').rangeBetween(-sys.maxsize, -1)
wsSpec_etf_dif_raw = Window.partitionBy('etf_id').orderBy('row_idx').rangeBetween(-sys.maxsize, 0)
def avg_list(p_list):
    #計算數字list的平均值
    return sum(p_list)/len(p_list)
#計算EMA的udf
def calculate_ema_native(close_p_list, window_len):
    #透過歷史收盤價計算
    if len(close_p_list) < window_len:
        return None
    elif len(close_p_list) == window_len:
        #if len of list = win_len then return avg, 
        return avg_list(close_p_list)
    else:
        #else EMA[t] =(EMA[t-1]*(win_len-1)+close[t]*2)/(win_len+1)
        ema = avg_list(close_p_list[:window_len])
        for price in close_p_list[window_len:]:
            ema = (ema*(window_len-1)+price*2)/(window_len+1)
        return ema
calculate_ema=udf(calculate_ema_native, DoubleType())
#計算BIAS的udf
def calculate_bias(close_p_list):
    #計算前日收盤價與N日均線之差比: (close price - MA)/MA   ,Paper 建議用20日MA
    #因要預測今日的收盤價，故計算前日收盤價與前20日均線
    if len(close_p_list) < 21:
        return None
    else:
        list_len = len(close_p_list)
        p_close = close_p_list[-1]
        cal_list = close_p_list[list_len-21: list_len-1]
        return p_close - avg_list(cal_list)
calculate_bias=udf(calculate_bias, DoubleType())

def get_min_max_last(p_list):
    #找出list中最大最小和最後一個值, 回傳(min, max, last)
    return (min(p_list), max(p_list), p_list[-1])
def calculate_raw_rsv(p_list):
    #RSV = (收盤價-9日低值)/(9日高值-9日低值)
    p_min, p_max, p_last = get_min_max_last(p_list)
    rsv = (p_last - p_min)/(p_max - p_min)
    return rsv
def calculate_rsv(p_9_list, k_prev, d_prev):
    #計算加權後的RSV，p_9_list=>9日收盤價
    rrsv = calculate_raw_rsv(p_9_list)
    k_curr = (1/3)*rrsv + (2/3)*k_prev
    d_curr = (1/3)*k_curr + (2/3)*d_prev
    return [k_curr, d_curr]
#計算隨機指標（Stochastic Oscillator，KD），原名%K&%D
def calculate_KD(close_p_list):
    win_len = 9 #看過去 9 日值
    #RSV = (收盤價-9日低值)/(9日高值-9日低值)
    #K_curr = 1/3*RSV + 2/3*K_prev
    #D_curr = 1/3*K_curr + 2/3*D_prev
    if len(close_p_list) < win_len:
        return None
    elif len(close_p_list) == win_len:
        #無前日K, D時，以0.5帶入
        return calculate_rsv(close_p_list, 0.5, 0.5)
    else:
        kds = calculate_rsv(close_p_list[0:9], 0.5, 0.5)
        for idx in range(1, (len(close_p_list)+1-9)):
            p_9_list = close_p_list[idx: idx+9]
            kds = calculate_rsv(p_9_list, kds[0], kds[1])
        return kds
calculate_KD=udf(calculate_KD, ArrayType(DoubleType()))

#計算差離值DIF = 12日EMA - 26日EMA
def calculate_DIF_native(close_p_list):
    if len(close_p_list) < 26:
        return None
    else:
        ema12 = calculate_ema_native(close_p_list, 12)
        ema26 = calculate_ema_native(close_p_list, 26)
        return ema12 - ema26
calculate_DIF=udf(calculate_DIF_native, DoubleType())

#計算MACD=(前一日MACD × (9 - 1) + 今日DIF × 2) ÷ (9 + 1)
def calculate_MACD(dif_list, dif_curr):
    win_len = 9
    if len(dif_list) < win_len:
        return None
    elif len(dif_list) == win_len:
        #if len of list = win_len then return avg, 
        return avg_list(dif_list)
    else:
        #MACD=(前一日MACD × (9 - 1) + 今日DIF × 2) ÷ (9 + 1)
        macd = avg_list(dif_list[:win_len])
        for price in dif_list[win_len:]:
            macd = (macd*(win_len-1)+dif_curr*2)/(win_len+1)
        return macd
calculate_MACD=udf(calculate_MACD, DoubleType())

#計算相對強弱指數(RSI)
def calculate_RSI(close_p_list):
    win_len = 9
    if len(close_p_list) < (win_len + 1):
        return None
    else:
        cur_list = close_p_list[1:]
        prv_list = close_p_list[0:-1]
        p_dif_list = list(map(lambda x,y : x - y, cur_list, prv_list)) #dif list
        u_list = []
        d_list = []
        for dif in p_dif_list:
            if dif == 0:
                #若兩天價格相同，則U及D皆等於零
                u_list.append(0)
                d_list.append(0)
            elif dif > 0:
                #在價格上升的日子, U = diff, D = 0
                u_list.append(dif)
                d_list.append(0)
            else:
                #在價格下跌的日子, U = 0, D = abs(diff)
                u_list.append(0)
                d_list.append(abs(dif))
        #RSI = ema(u,9)/(ema(u,9)+ema(d,9))
        ema_u = calculate_ema_native(u_list, win_len)
        ema_d = calculate_ema_native(d_list, win_len)
        return ema_u/(ema_u + ema_d)
calculate_RSI=udf(calculate_RSI, DoubleType())

#計算威廉指標（Williams %R）
def calculate_WR(close_p_list):
    win_len = 9
    if len(close_p_list) < win_len:
        return None
    else:
        p_list = close_p_list[len(close_p_list) - win_len :]
        return 1.0 - calculate_raw_rsv(p_list)
calculate_WR=udf(calculate_WR, DoubleType())


In [38]:
#calculate ema [5,10,20] #cannot remove row_idx, row_idx for next window usage
tetfp_dt2=tetfp_dt.withColumn("row_idx", rank().over(wsSpec_etf)) \
    .withColumn("close_price_raw", collect_list(col('etf_close')).over(wsSpec_etf_close_price_raw)) \
    .withColumn("EMA5", calculate_ema(col("close_price_raw"), lit(5))) \
    .withColumn("EMA10", calculate_ema(col("close_price_raw"), lit(10))) \
    .withColumn("EMA20", calculate_ema(col("close_price_raw"), lit(20))) \
    .withColumn("BIAS", calculate_bias(col("close_price_raw"))) \
    .withColumn("KD", calculate_KD(col("close_price_raw"))) \
    .withColumn("K", col("KD")[0]).withColumn("D", col("KD")[1]) \
    .withColumn("DIF", calculate_DIF(col("close_price_raw"))) \
    .withColumn("dif_list", collect_list(col('DIF')).over(wsSpec_etf_dif_raw)) \
    .withColumn("MACD", calculate_MACD(col("dif_list"), col("DIF"))) \
    .withColumn("RSI", calculate_RSI(col("close_price_raw")))\
    .withColumn("WR", calculate_WR(col("close_price_raw")))

tetfp_dt2.cache()
tetfp_dt2.printSchema()

root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- etf_name: string (nullable = true)
 |-- etf_open: double (nullable = true)
 |-- etf_high: double (nullable = true)
 |-- etf_low: double (nullable = true)
 |-- etf_close: double (nullable = true)
 |-- etf_count: double (nullable = true)
 |-- row_idx: integer (nullable = true)
 |-- close_price_raw: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- EMA5: double (nullable = true)
 |-- EMA10: double (nullable = true)
 |-- EMA20: double (nullable = true)
 |-- BIAS: double (nullable = true)
 |-- KD: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- K: double (nullable = true)
 |-- D: double (nullable = true)
 |-- DIF: double (nullable = true)
 |-- dif_list: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- MACD: double (nullable = true)
 |-- RSI: double (nullable = true)
 |-- WR: double (nullable = true)



In [134]:
from pyspark.ml.feature import MinMaxScaler, StandardScaler
from pyspark.ml.linalg import Vectors
tot_dt = tetfp_dt2.filter("MACD is not null") \
    .select("etf_id", "etf_date", "row_idx", "EMA5", "EMA10", "EMA20", "BIAS", "K", "D", "DIF", "MACD", "RSI", "WR", "etf_close") \
    .orderBy("etf_id", "etf_date", ascending=True)
    
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
#將Feature合併為Vector 並作標準化
featureCols = ["EMA5", "EMA10", "EMA20", "BIAS", "K", "D", "DIF", "MACD", "RSI", "WR"]
assembler = VectorAssembler(
    inputCols=featureCols,
    outputCol="features")
tot_dt_1 = assembler.transform(tot_dt)
minmax_scaler = MinMaxScaler(inputCol="features", outputCol="stdFeatures")
scaler_model = minmax_scaler.fit(tot_dt_1)
#std_scaler = StandardScaler(inputCol="features", outputCol="stdFeatures")
#scaler_model = std_scaler.fit(tot_dt_1)
tot_dt_scale = scaler_model.transform(tot_dt_1)
#tot_dt_scale=tot_dt_1.withColumn("stdFeatures", col("features")) #測試不作標準化

In [149]:
from pyspark.mllib.linalg import DenseVector

a = DenseVector([1.0,2.0,3.0,4.0,5.0])

len(a)

5

In [159]:
def my_to_list(vec):
    return vec
    #mylist = []
    #for idx in range(0, len(vec)):
    #    mylist.append(vec[idx])
    #return mylist
#my_to_list=udf(my_to_list, ArrayType(DoubleType()))
my_to_list=udf(my_to_list)

tot_dt_scale2 = tot_dt_scale.select(["etf_id", "etf_date", "stdFeatures"]).withColumn("stdFeatureList", my_to_list(col("stdFeatures")))
#for cidx in range(0, len(featureCols)):
#    tot_dt_scale2 = tot_dt_scale2.withColumn(featureCols[cidx], col("stdFeatureList")[cidx])
#tot_dt_scale2=tot_dt_scale2.drop("stdFeatures","stdFeatureList")
tot_dt_scale2.printSchema()
tot_dt_scale2.show(10,False)

root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- stdFeatures: vector (nullable = true)
 |-- stdFeatureList: string (nullable = true)



Py4JJavaError: An error occurred while calling o2112.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 495.0 failed 1 times, most recent failure: Lost task 0.0 in stage 495.0 (TID 25440, localhost, executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for pyspark.ml.linalg.DenseVector)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:707)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:175)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:99)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:112)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec$$anonfun$evaluate$1.apply(BatchEvalPythonExec.scala:86)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec$$anonfun$evaluate$1.apply(BatchEvalPythonExec.scala:85)
	at scala.collection.Iterato

In [40]:
###### 取出4/16~4/27 (共兩週資料作為測試集)
train_dt = tot_dt_scale.filter("etf_date < '" + predict_start_date + "' and MACD is not null") \
    .select("etf_id", "etf_date", "row_idx", "stdFeatures", "etf_close") \
    .orderBy("etf_id", "etf_date", ascending=True)
test_dt = tot_dt_scale.filter("etf_date >= '" + predict_start_date + "'") \
    .select("etf_id", "etf_date", "row_idx", "stdFeatures", "etf_close") \
    .orderBy("etf_id", "etf_date", ascending=True)
print('train count: ', str(train_dt.count()), ', test count: ', str(test_dt.count()))
train_dt.show(10)
test_dt.show(10)

train count:  18442 , test count:  72
+-------+--------+-------+--------------------+---------+
| etf_id|etf_date|row_idx|         stdFeatures|etf_close|
+-------+--------+-------+--------------------+---------+
|0050   |20130227|     35|[0.58940415111937...|     55.2|
|0050   |20130301|     36|[0.58819098072642...|     55.4|
|0050   |20130304|     37|[0.58824157226986...|    54.75|
|0050   |20130305|     38|[0.58548234159790...|     55.2|
|0050   |20130306|     39|[0.58557644104544...|    55.45|
|0050   |20130307|     40|[0.58671338876723...|     55.4|
|0050   |20130308|     41|[0.58725651096374...|     55.8|
|0050   |20130311|     42|[0.58933733603890...|     55.9|
|0050   |20130312|     43|[0.59115423865838...|    55.55|
|0050   |20130313|     44|[0.59086160641190...|    55.65|
+-------+--------+-------+--------------------+---------+
only showing top 10 rows

+-------+--------+-------+--------------------+---------+
| etf_id|etf_date|row_idx|         stdFeatures|etf_close|
+-------

In [41]:
#test_dt.show(20, False)

In [42]:
train_dt.cache()
test_dt.cache()

DataFrame[etf_id: string, etf_date: string, row_idx: int, stdFeatures: vector, etf_close: double]

In [43]:
#取出etf的distinct id
etf_ids = []
for row in train_dt.select("etf_id").distinct().collect():
    etf_ids.append(row["etf_id"])
etf_ids

['00701  ',
 '0051   ',
 '0057   ',
 '006203 ',
 '0052   ',
 '0050   ',
 '0055   ',
 '0054   ',
 '0059   ',
 '00690  ',
 '00713  ',
 '006204 ',
 '006208 ',
 '0053   ',
 '006201 ',
 '0056   ',
 '00692  ',
 '0058   ']

In [44]:
#prepare pandas for xgb
train_pd_map = {}
for etfid in etf_ids:
    train_data = train_dt.filter("etf_id='" + etfid + "'")
    train_pd = train_data.toPandas()
    print(etfid, ': feature length: ', len(train_pd))
    train_pd_map.update({etfid: train_pd})

00701   : feature length:  136
0051    : feature length:  1269
0057    : feature length:  1269
006203  : feature length:  1269
0052    : feature length:  1269
0050    : feature length:  1269
0055    : feature length:  1269
0054    : feature length:  1269
0059    : feature length:  1269
00690   : feature length:  231
00713   : feature length:  107
006204  : feature length:  1269
006208  : feature length:  1269
0053    : feature length:  1269
006201  : feature length:  1269
0056    : feature length:  1270
00692   : feature length:  201
0058    : feature length:  1269


In [131]:
pd = train_pd_map['00690  ']
print('features: ', len(pd[["stdFeatures"]].values), pd[["stdFeatures"]].values.shape)
print('labels: ', len(pd["etf_close"].values), pd.etf_close.values.shape)
ll = pd[["etf_id","etf_close"]].values
ll
type(ll[0][1])

features:  231 (231, 1)
labels:  231 (231,)


float

In [132]:
ll = []
for x in pd["stdFeatures"]:
    ll.append(x.values)
ll
type(ll[0][0])
#type(pd["stdFeatures"].values)
#pd["stdFeatures"][0].values
#pd["stdFeatures"].values

numpy.float64

In [78]:
#訓練Model及評估(XGBRegressor - etf_id wise) 
import xgboost as xgb

model_map = {}
for etfid in etf_ids:
    rf_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75, 
                                 colsample_bytree=1, max_depth=10, seed=8240, n_jobs=4)
    train_x = train_pd_map[etfid][["stdFeatures"]].values
    train_y = train_pd_map[etfid]["etf_close"]
    rf_model.fit(train_x, train_y)
    model_map.update({etfid: rf_model})

ValueError: setting an array element with a sequence.

In [47]:
#find last records of all etf to be the base of next record
tetf_dt_prod = tetfp_dt2.filter("etf_date < '" + predict_start_date + "' and MACD is not null")
tetf_max_idx = tetf_dt_prod.groupBy("etf_id").max("row_idx")
tetf_max = tetf_max_idx.select(col("etf_id"), col("max(row_idx)").cast("Double").alias("row_idx")) \
    .join(tetf_dt_prod, ["etf_id", "row_idx"], "inner") \
    .select("etf_id", "etf_date", "row_idx", "etf_close", "close_price_raw", "dif_list") \
    .orderBy('etf_id')

tetf_max.show()

+-------+--------+-------+---------+--------------------+--------------------+
| etf_id|etf_date|row_idx|etf_close|     close_price_raw|            dif_list|
+-------+--------+-------+---------+--------------------+--------------------+
|0050   |20180427| 1303.0|     79.2|[54.4, 54.85, 54....|[0.35882671868262...|
|0051   |20180427| 1303.0|    32.11|[26.09, 26.12, 26...|[0.23810818793990...|
|0052   |20180427| 1303.0|     53.2|[32.72, 32.12, 32...|[0.30196772810150...|
|0053   |20180427| 1303.0|     34.2|[23.26, 23.11, 22...|[0.15597485321855...|
|0054   |20180427| 1303.0|    23.09|[19.4, 19.37, 19....|[0.09327655672724...|
|0055   |20180427| 1303.0|    17.04|[11.47, 11.49, 11...|[0.31374969126963...|
|0056   |20180427| 1304.0|    25.15|[22.95, 23.06, 22...|[0.31828091002715...|
|0057   |20180427| 1303.0|    48.74|[31.94, 31.93, 31...|[0.20109705259728...|
|0058   |20180427| 1303.0|    45.06|[32.12, 31.96, 32...|[-0.1346882469799...|
|0059   |20180427| 1303.0|    41.97|[24.65, 24.67, 2

In [48]:
#建立產生feature之方法
def create_feature(orig_dt):
    new_dt = orig_dt.withColumn("EMA5", calculate_ema(col("close_price_raw"), lit(5))) \
    .withColumn("EMA10", calculate_ema(col("close_price_raw"), lit(10))) \
    .withColumn("EMA20", calculate_ema(col("close_price_raw"), lit(20))) \
    .withColumn("BIAS", calculate_bias(col("close_price_raw"))) \
    .withColumn("KD", calculate_KD(col("close_price_raw"))) \
    .withColumn("K", col("KD")[0]).withColumn("D", col("KD")[1]) \
    .withColumn("MACD", calculate_MACD(col("dif_list"), col("DIF"))) \
    .withColumn("RSI", calculate_RSI(col("close_price_raw")))\
    .withColumn("WR", calculate_WR(col("close_price_raw")))
    return new_dt
#計算上或下的值(udf)
def judge_up_down_pred(curr_price, close_price_list):
    if len(close_price_list) == 0:
        return 0.0
    else:
        prev_price = close_price_list[-1]
        if curr_price == prev_price:
            return 0.0
        elif curr_price > prev_price:
            return 1.0
        else:
            return 2.0
judge_up_down_pred=udf(judge_up_down_pred, DoubleType())
#建立預測方法
def doPredict(test_dt, etf_ids, model_dic):
    predict_res = None
    for etfid in etf_ids:
        test_data = test_dt.filter("etf_id='" + etfid + "'")
        predicts = model_dic[etfid].transform(test_data)
        if predict_res is None:
            predict_res = predicts
        else:
            predict_res = predict_res.unionAll(predicts)
    predict_res2 = predict_res.withColumn("pred_ud", judge_up_down_pred(col("prediction"), col("close_price_raw")))
    return predict_res2

In [49]:
schema = tetfp_dt2.select("etf_id", "etf_date", col("row_idx").cast("Double"), "close_price_raw", "DIF", "dif_list").schema
idx_plus = 0.0
predict_res_final = None
predict_range = [d for d in next_date_range if d not in ignore_dates]
for date in predict_range:
    next_rows = []
    idx_plus = idx_plus +1
    for row in tetf_max.collect():
        close_price_raw = row["close_price_raw"]
        close_price_raw.append(row["etf_close"])
        dif_list = row["dif_list"]
        dif = calculate_DIF_native(close_price_raw)
        dif_list.append(dif)
        next_row = (row["etf_id"], date, row["row_idx"]+idx_plus, close_price_raw, dif ,dif_list)
        next_rows.append(next_row)
    orig_dt = spark.createDataFrame(next_rows, schema)
    new_dt = create_feature(orig_dt) #加入預測用的feature
    feature_dt = assembler.transform(new_dt).withColumn("stdFeatures", col("features")) #作出 FeatureVector
    #feature_dt.select("etf_id", "etf_date", "stdFeatures").show(10, False)
    #do predict using existed model
    pred_res = doPredict(feature_dt, etf_ids, model_map)
    #取出所有預測結果作合併，以進行後續成績計算
    merged_dt = pred_res.select("etf_id", "etf_date", "prediction", "pred_ud")
    if predict_res_final is None:
        predict_res_final = merged_dt
    else:
        predict_res_final = predict_res_final.unionAll(merged_dt)
    tetf_max = pred_res.select("etf_id", "etf_date", "row_idx", col("prediction").alias("etf_close"), "close_price_raw", "dif_list")
    
predict_res_final.cache()
predict_res_final.show(10)


NameError: name 'model_map' is not defined

In [ ]:
# 評估Model的RMES
####### with min-max scaler (depth=10, bin=32) #########
#(RMSE) on test data = 1.63371   accuracy = 0.555556 final score:  11.370977902160591
####### with min-max scaler (depth=12,10, bin=48) #########
#(RMSE) on test data = 1.61946   accuracy = 0.527778 final score:  11.125999693476427
####### with min-max scaler (depth=10, bin=30) #########
#(RMSE) on test data = 1.63371   accuracy = 0.48 final score:  10.00

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator, MulticlassClassificationEvaluator
if DoEval:
    tetf_dt_eval = tetfp_dt2.filter("etf_date >= '" + predict_start_date + "'") \
        .select("etf_id", "etf_date", "etf_close", "close_price_raw")
    eval_dt = predict_res_final.join(tetf_dt_eval, ["etf_id", "etf_date"], "inner") \
        .withColumn("act_ud", judge_up_down_pred(col("etf_close"), col("close_price_raw")))
    eval_dt.cache()
    #eval_dt.orderBy("etf_id", "etf_date", ascending=True).show(10)

    evaluator = RegressionEvaluator(
        labelCol="etf_close", predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(eval_dt)
    print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
    evaluator = MulticlassClassificationEvaluator(
        labelCol="act_ud", predictionCol="pred_ud", metricName="accuracy")
    accuracy = evaluator.evaluate(eval_dt)
    print("accuracy = %g " % accuracy)
    
    #define method to evaluate by ETF way
    final_res = eval_dt.select("etf_id", "etf_date", "prediction", "pred_ud", "etf_close", "act_ud") \
        .orderBy("etf_date", "etf_id").collect()
    weights = [0.1, 0.15, 0.2, 0.25, 0.3]
    judge_score = 0.0
    for row in final_res:
        #(up equal*0.5+((act_p - abs(pred_p - act_p))/act_p)*0.5)*weight
        act_price = row["etf_close"]
        pred_price = row["prediction"]
        eidx = next_date_range.index(row["etf_date"])
        judge_score = judge_score + \
            ((0.5 if row["pred_ud"] == row["act_ud"] else 0) + \
             ((act_price-abs(pred_price-act_price))/act_price)*0.5)*weights[eidx]
    print("final score: ", judge_score)
#numtree

In [ ]:
final_res = predict_res_final.orderBy("etf_id", "etf_date").collect()

In [ ]:
#export to pandas
etf_ids = []
mon_ud = []
mon_price = [] 
tue_ud = []
tue_price = []
wed_ud = []
wed_price = []
thu_ud = []
thu_price = []
fri_ud = []
fri_price = []

def encode_ud(oper_ud):
    if oper_ud == 0.0:
        return 0
    elif oper_ud == 1.0:
        return 1
    else:
        return -1

for row in final_res:
    etf_id = row["etf_id"]
    if etf_id not in etf_ids:
        etf_ids.append(etf_id)
    etf_date = row["etf_date"]
    eidx = next_date_range.index(row["etf_date"])
    if eidx == 0:
        mon_ud.append(encode_ud(row["pred_ud"]))
        mon_price.append(row["prediction"])
    elif eidx == 1:
        tue_ud.append(encode_ud(row["pred_ud"]))
        tue_price.append(row["prediction"])
    elif eidx == 2:
        wed_ud.append(encode_ud(row["pred_ud"]))
        wed_price.append(row["prediction"])
    elif eidx == 3:
        thu_ud.append(encode_ud(row["pred_ud"]))
        thu_price.append(row["prediction"])
    elif eidx == 4:
        fri_ud.append(encode_ud(row["pred_ud"]))
        fri_price.append(row["prediction"])
        
if len(mon_ud) == 0:
    mon_ud = list(0 for i in range(0,len(etf_ids)))
    mon_price = list(0.0 for i in range(0,len(etf_ids)))
if len(tue_ud) == 0:
    tue_ud = list(0 for i in range(0,len(etf_ids)))
    tue_price = list(0.0 for i in range(0,len(etf_ids)))
if len(wed_ud) == 0:
    wed_ud = list(0 for i in range(0,len(etf_ids)))
    wed_price = list(0.0 for i in range(0,len(etf_ids)))
if len(thu_ud) == 0:
    thu_ud = list(0 for i in range(0,len(etf_ids)))
    thu_price = list(0.0 for i in range(0,len(etf_ids)))
if len(fri_ud) == 0:
    fri_ud = list(0 for i in range(0,len(etf_ids)))
    fri_price = list(0.0 for i in range(0,len(etf_ids)))
    
dic = {"ETFid": etf_ids, 
       "Mon_ud": mon_ud, "Mon_cprice": mon_price,
       "Tue_ud": tue_ud, "Tue_cprice": tue_price,
       "Wed_ud": wed_ud, "Wed_cprice": wed_price,
       "Thu_ud": thu_ud, "Thu_cprice": thu_price,
       "Fri_ud": fri_ud, "Fri_cprice": fri_price
      }
final_df = pd.DataFrame(data=dic)[['ETFid','Mon_ud','Mon_cprice','Tue_ud','Tue_cprice',
                                  'Wed_ud','Wed_cprice','Thu_ud','Thu_cprice',
                                  'Fri_ud','Fri_cprice']]
final_df

In [ ]:
final_df.to_csv(Path.replace("file:","") + "/etf_price_pred.csv",index=False)

In [ ]:
#ll = [46.92, 47.31, 47.0, 46.79, 46.49, 46.66, 47.0, 46.96, 47.0]
print(list(0 for i in range(0,5)))
ll = [46.92, 47.31, 47.0, 46.79, 46.49, 46.66]
win_len=5
print(ll[0])
print(ll[0: -1])
print(list(map(lambda x,y : x - y, ll[1:], ll[0: -1])))
for x in ll[win_len:]:
    print(x)
ema = sum(ll[:win_len])/len(ll[:win_len])
print(ema)
for price in ll[win_len:]:
    ema = (ema*(win_len-1)+price*2)/(win_len+1)
tup1, tup2 = (1,2)
print(tup1, ' ', tup2)
tup = (3,4)
print(tup[0], ' ', tup[1])
list(range(0,2))
test_dic = {}
test_dic.update({"0051": assembler})
print(test_dic['0051'])
print([x for x in next_date_range if x not in ignore_dates])